In [91]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
pd.options.display.max_columns = None

# find the player id with name

In [92]:
from nba_api.stats.static import players

#players we want to look into
superstars = ["Kevin Durant", "Luka Doncic"]

#initialize empty lists that will put player id into
player_ids = []
for i in superstars:
    player_ids.append(players.find_players_by_full_name(i)[0]['id'])

In [93]:
player = dict(zip(superstars,player_ids))

In [94]:
p = player["Luka Doncic"]

In [95]:
from nba_api.stats.endpoints import shotchartdetail
seasons =['2018-19', '2019-20', '2020-21', '2021-22', '2022-23']
player_id_ = p
shots = []
for i in seasons:
    shots.append(shotchartdetail.ShotChartDetail(team_id=0,
    player_id=player_id_,
    season_nullable= i,
    context_measure_simple = 'FGA',
    season_type_all_star='Regular Season').get_data_frames()[0])

player_data = dict(zip(seasons,shots))

for df in player_data.values():     
    df.drop(['GRID_TYPE','EVENT_TYPE','ACTION_TYPE'], inplace=True, axis=1)

In [96]:
df_18_19 = player_data['2018-19']
df_19_20 = player_data['2019-20']
df_20_21 = player_data['2020-21']
df_21_22 = player_data['2021-22']
df_22_23 = player_data['2022-23']

In [97]:
df_18_19['SHOT_MADE_FLAG'].unique()

array([0, 1])

In [98]:
df_18_19.groupby('SHOT_ZONE_RANGE').sum()['SHOT_ATTEMPTED_FLAG']

SHOT_ZONE_RANGE
16-24 ft.           51
24+ ft.            498
8-16 ft.           182
Back Court Shot     15
Less Than 8 ft.    440
Name: SHOT_ATTEMPTED_FLAG, dtype: int64

In [99]:
def shot_percentage(df_sub):
    d = (df_sub['SHOT_MADE_FLAG'].sum() / df_sub['SHOT_ATTEMPTED_FLAG'].sum())*100
    return d

acc = df_18_19.groupby('SHOT_ZONE_RANGE').apply(shot_percentage)

In [100]:
acc

SHOT_ZONE_RANGE
16-24 ft.          41.176471
24+ ft.            33.534137
8-16 ft.           39.560440
Back Court Shot     6.666667
Less Than 8 ft.    55.681818
dtype: float64

# drawing basketball court with plotly

In [125]:
def draw_plotly_court(fig, fig_width=600, margins=10):

    import numpy as np
        
    # From: https://community.plot.ly/t/arc-shape-with-path/7205/5
    def ellipse_arc(x_center=0.0, y_center=0.0, a=10.5, b=10.5, start_angle=0.0, end_angle=2 * np.pi, N=200, closed=False):
        t = np.linspace(start_angle, end_angle, N)
        x = x_center + a * np.cos(t)
        y = y_center + b * np.sin(t)
        path = f'M {x[0]}, {y[0]}'
        for k in range(1, len(t)):
            path += f'L{x[k]}, {y[k]}'
        if closed:
            path += ' Z'
        return path

    fig_height = fig_width * (470 + 2 * margins) / (500 + 2 * margins)
    fig.update_layout(width=fig_width, height=fig_height)

    # Set axes ranges
    fig.update_xaxes(range=[-250 - margins, 250 + margins])
    fig.update_yaxes(range=[-52.5 - margins, 417.5 + margins])

    threept_break_y = 89.47765084
    three_line_col = "#777777"
    main_line_col = "#777777"

    fig.update_layout(
        # Line Horizontal
        margin=dict(l=20, r=20, t=20, b=20),
        paper_bgcolor="white",
        plot_bgcolor="white",
        yaxis=dict(
            scaleanchor="x",
            scaleratio=1,
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False,
            fixedrange=True,
        ),
        xaxis=dict(
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False,
            fixedrange=True,
        ),
        shapes=[
            dict(
                type="rect", x0=-250, y0=-52.5, x1=250, y1=417.5,
                line=dict(color=main_line_col, width=1),
                # fillcolor='#333333',
                layer='below'
            ),
            dict(
                type="rect", x0=-80, y0=-52.5, x1=80, y1=137.5,
                line=dict(color=main_line_col, width=1),
                # fillcolor='#333333',
                layer='below'
            ),
            dict(
                type="rect", x0=-60, y0=-52.5, x1=60, y1=137.5,
                line=dict(color=main_line_col, width=1),
                # fillcolor='#333333',
                layer='below'
            ),
            dict(
                type="circle", x0=-60, y0=77.5, x1=60, y1=197.5, xref="x", yref="y",
                line=dict(color=main_line_col, width=1),
                # fillcolor='#dddddd',
                layer='below'
            ),
            dict(
                type="line", x0=-60, y0=137.5, x1=60, y1=137.5,
                line=dict(color=main_line_col, width=1),
                layer='below'
            ),

            dict(
                type="rect", x0=-2, y0=-7.25, x1=2, y1=-12.5,
                line=dict(color="#ec7607", width=1),
                fillcolor='#ec7607',
            ),
            dict(
                type="circle", x0=-7.5, y0=-7.5, x1=7.5, y1=7.5, xref="x", yref="y",
                line=dict(color="#ec7607", width=1),
            ),
            dict(
                type="line", x0=-30, y0=-12.5, x1=30, y1=-12.5,
                line=dict(color="#ec7607", width=1),
            ),

            dict(type="path",
                 path=ellipse_arc(a=40, b=40, start_angle=0, end_angle=np.pi),
                 line=dict(color=main_line_col, width=1), layer='below'),
            dict(type="path",
                 path=ellipse_arc(a=237.5, b=237.5, start_angle=0.386283101, end_angle=np.pi - 0.386283101),
                 line=dict(color=main_line_col, width=1), layer='below'),
            dict(
                type="line", x0=-220, y0=-52.5, x1=-220, y1=threept_break_y,
                line=dict(color=three_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-220, y0=-52.5, x1=-220, y1=threept_break_y,
                line=dict(color=three_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=220, y0=-52.5, x1=220, y1=threept_break_y,
                line=dict(color=three_line_col, width=1), layer='below'
            ),

            dict(
                type="line", x0=-250, y0=227.5, x1=-220, y1=227.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=250, y0=227.5, x1=220, y1=227.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=17.5, x1=-80, y1=17.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=27.5, x1=-80, y1=27.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=57.5, x1=-80, y1=57.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=87.5, x1=-80, y1=87.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=17.5, x1=80, y1=17.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=27.5, x1=80, y1=27.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=57.5, x1=80, y1=57.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=87.5, x1=80, y1=87.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),

            dict(type="path",
                 path=ellipse_arc(y_center=417.5, a=60, b=60, start_angle=-0, end_angle=-np.pi),
                 line=dict(color=main_line_col, width=1), layer='below'),

        ]
    )
    return True

# Make the graph

In [102]:
xlocs = df_22_23['LOC_X']
ylocs = df_22_23['LOC_Y']
color = df_22_23['SHOT_MADE_FLAG']


fig = px.scatter(x=xlocs, y=ylocs,
                color=color,color_discrete_map= ['green','red'],
                symbol = color,symbol_sequence= ['circle', 'x'])
fig.update_traces(marker=dict(size=9,line=dict(width=1)))
draw_plotly_court(fig)
fig.update_yaxes(visible=False)
fig.update_xaxes(visible = False)
fig.update_layout(coloraxis_showscale=False,legend=dict(orientation='h',yanchor="top",y=0,xanchor="left",x=0.67))
fig.show(config=dict(displayModeBar=False))

In [19]:
import panel as pn
from panel.interact import interact, interactive, fixed, interact_manual
from panel import widgets
pn.extension()

In [25]:

def shots_graph(df=df_22_23):
    fig = px.scatter(x=df['LOC_X'], y=df['LOC_Y'],
                    color=df['SHOT_MADE_FLAG'],color_discrete_map= ['green','red'],
                    symbol = df['SHOT_MADE_FLAG'],symbol_sequence= ['circle', 'x'])
    fig.update_traces(marker=dict(size=9,line=dict(width=1)))
    draw_plotly_court(fig)
    fig.update_yaxes(visible=False)
    fig.update_xaxes(visible = False)
    fig.update_layout(coloraxis_showscale=False,legend=dict(orientation='h',yanchor="top",y=0,xanchor="left",x=0.67))
    return fig.show(config=dict(displayModeBar=False))

select = pn.widgets.Select(name='Select season', options=['df_18_19', 'df_19_20', 'df_21_22'])
output = pn.bind(shots_graph,select)

In [129]:
a = shots_graph(df_21_22)

In [130]:
img_player = pn.pane.PNG('/Users/giorgio/Desktop/python/nba_shots/Luka Dončić.png', width=200)
select = pn.widgets.Select(name='Select season', options=['df_18_19', 'df_19_20', 'df_20_21'])
dashboard = pn.Column(a)

    
Case_dash = pn.template.BootstrapTemplate(
title='Shots of Luka Doncic',
main= dashboard,
accent_base_color="#88d8b0",
header_background="#00538C")

Case_dash.show({'displayModeBar': True})

Launching server at http://localhost:57329
